In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query

from datetime import datetime

종목코드열람

In [2]:
shcode_list = query.query_shcode_list_today()
today = datetime.now().strftime("%Y%m%d")

In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [5]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 500
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 5:
            average_volume_test[ind] = 1
            
    return average_volume_test

#최종

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    #daily_data = query.query_ohlcv(v, to = today)
    daily_data = query.query_ohlcv(v, fr = '20181201',to = today)
    week_data = query.query_ohlcv_t1102(shcode = v, to = today)
    daily_data = daily_data.append(week_data).reset_index(drop = True)
    array_daily = np.array(daily_data, dtype = np.int64)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v4' +'.csv'
result.to_csv(filename, sep=',')

In [6]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.05
volume_money = 20000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    #daily_data = query.query_ohlcv(v, to = today)
    daily_data = query.query_ohlcv(v, fr = '20181201',to = today)
    week_data = query.query_ohlcv_t1102(shcode = v, to = today)
    daily_data = daily_data.append(week_data).reset_index(drop = True)
    array_daily = np.array(daily_data, dtype = np.int64)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v4' +'.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-12-5_13:12:49 : , 0 , 000020 , 동화약품 -------------------  
O_1. ratio_test 통과 0 , 000020 , 동화약품
X_2. total_volume_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-12-5_13:12:50 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-12-5_13:12:50 : , 2 , 000050 , 경방 -------------------  
O_1. ratio_test 통과 2 , 000050 , 경방
X_2. total_volume_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-12-5_13:12:51 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-12-5_13:12:51 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-12-5_13:12:52 : , 5 , 000075 , 삼양홀딩스우 -------------------  
O_1. ratio_test 통과 5 , 000075 , 삼양홀딩스우
X_2. total_volume_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-12-5_13:12:53 : , 6 , 000080 , 하이트진로 -------------------  
O_

X_1. ratio_test 없음 41 , 000650 , 천일고속
------------------- test 시작_2019-12-5_13:13:17 : , 42 , 000660 , SK하이닉스 -------------------  
O_1. ratio_test 통과 42 , 000660 , SK하이닉스
O_2. total_volume_test 통과 42 , 000660 , SK하이닉스
X_3. average_volume_test 없음 42 , 000660 , SK하이닉스
------------------- test 시작_2019-12-5_13:13:18 : , 43 , 000670 , 영풍 -------------------  
O_1. ratio_test 통과 43 , 000670 , 영풍
O_2. total_volume_test 통과 43 , 000670 , 영풍
X_3. average_volume_test 없음 43 , 000670 , 영풍
------------------- test 시작_2019-12-5_13:13:19 : , 44 , 000680 , LS네트웍스 -------------------  
O_1. ratio_test 통과 44 , 000680 , LS네트웍스
X_2. total_volume_test 없음 44 , 000680 , LS네트웍스
------------------- test 시작_2019-12-5_13:13:19 : , 45 , 000700 , 유수홀딩스 -------------------  
O_1. ratio_test 통과 45 , 000700 , 유수홀딩스
X_2. total_volume_test 없음 45 , 000700 , 유수홀딩스
------------------- test 시작_2019-12-5_13:13:20 : , 46 , 000720 , 현대건설 -------------------  
O_1. ratio_test 통과 46 , 000720 , 현대건설
O_2. total_volume_test 통과 46 

O_1. ratio_test 통과 83 , 001360 , 삼성제약
O_2. total_volume_test 통과 83 , 001360 , 삼성제약
O_3. average_volume_test 통과 83 , 001360 , 삼성제약
------------------- test 완료 : , 83 , 001360 , 삼성제약 ------------------- 
------------------- test 시작_2019-12-5_13:13:41 : , 84 , 001380 , SG충방 -------------------  
O_1. ratio_test 통과 84 , 001380 , SG충방
O_2. total_volume_test 통과 84 , 001380 , SG충방
O_3. average_volume_test 통과 84 , 001380 , SG충방
------------------- test 완료 : , 84 , 001380 , SG충방 ------------------- 
------------------- test 시작_2019-12-5_13:13:41 : , 85 , 001390 , KG케미칼 -------------------  
O_1. ratio_test 통과 85 , 001390 , KG케미칼
X_2. total_volume_test 없음 85 , 001390 , KG케미칼
------------------- test 시작_2019-12-5_13:13:42 : , 86 , 001420 , 태원물산 -------------------  
O_1. ratio_test 통과 86 , 001420 , 태원물산
X_2. total_volume_test 없음 86 , 001420 , 태원물산
------------------- test 시작_2019-12-5_13:13:42 : , 87 , 001430 , 세아베스틸 -------------------  
O_1. ratio_test 통과 87 , 001430 , 세아베스틸
X_2. total_volume_t

O_1. ratio_test 통과 124 , 002025 , 코오롱우
X_2. total_volume_test 없음 124 , 002025 , 코오롱우
------------------- test 시작_2019-12-5_13:14:3 : , 125 , 002030 , 아세아 -------------------  
X_1. ratio_test 없음 125 , 002030 , 아세아
------------------- test 시작_2019-12-5_13:14:4 : , 126 , 002070 , 남영비비안 -------------------  
O_1. ratio_test 통과 126 , 002070 , 남영비비안
O_2. total_volume_test 통과 126 , 002070 , 남영비비안
O_3. average_volume_test 통과 126 , 002070 , 남영비비안
------------------- test 완료 : , 126 , 002070 , 남영비비안 ------------------- 
------------------- test 시작_2019-12-5_13:14:4 : , 127 , 002100 , 경농 -------------------  
O_1. ratio_test 통과 127 , 002100 , 경농
O_2. total_volume_test 통과 127 , 002100 , 경농
O_3. average_volume_test 통과 127 , 002100 , 경농
------------------- test 완료 : , 127 , 002100 , 경농 ------------------- 
------------------- test 시작_2019-12-5_13:14:5 : , 128 , 002140 , 고려산업 -------------------  
O_1. ratio_test 통과 128 , 002140 , 고려산업
O_2. total_volume_test 통과 128 , 002140 , 고려산업
O_3. average_volum

O_1. ratio_test 통과 161 , 002810 , 삼영무역
X_2. total_volume_test 없음 161 , 002810 , 삼영무역
------------------- test 시작_2019-12-5_13:14:22 : , 162 , 002820 , 선창산업 -------------------  
X_1. ratio_test 없음 162 , 002820 , 선창산업
------------------- test 시작_2019-12-5_13:14:23 : , 163 , 002840 , 미원상사 -------------------  
O_1. ratio_test 통과 163 , 002840 , 미원상사
X_2. total_volume_test 없음 163 , 002840 , 미원상사
------------------- test 시작_2019-12-5_13:14:24 : , 164 , 002870 , 신풍제지 -------------------  
O_1. ratio_test 통과 164 , 002870 , 신풍제지
X_2. total_volume_test 없음 164 , 002870 , 신풍제지
------------------- test 시작_2019-12-5_13:14:24 : , 165 , 002880 , 대유에이텍 -------------------  
O_1. ratio_test 통과 165 , 002880 , 대유에이텍
O_2. total_volume_test 통과 165 , 002880 , 대유에이텍
X_3. average_volume_test 없음 165 , 002880 , 대유에이텍
------------------- test 시작_2019-12-5_13:14:25 : , 166 , 002900 , 동양물산 -------------------  
O_1. ratio_test 통과 166 , 002900 , 동양물산
O_2. total_volume_test 통과 166 , 002900 , 동양물산
O_3. average_volume

...

O_1. ratio_test 통과 2788 , 255440 , 야스
O_2. total_volume_test 통과 2788 , 255440 , 야스
X_3. average_volume_test 없음 2788 , 255440 , 야스
------------------- test 시작_2019-12-5_13:38:32 : , 2789 , 256150 , 한독크린텍 -------------------  
X_1. ratio_test 없음 2789 , 256150 , 한독크린텍
------------------- test 시작_2019-12-5_13:38:32 : , 2790 , 256630 , 포인트엔지니어링 -------------------  
O_1. ratio_test 통과 2790 , 256630 , 포인트엔지니어링
O_2. total_volume_test 통과 2790 , 256630 , 포인트엔지니어링
O_3. average_volume_test 통과 2790 , 256630 , 포인트엔지니어링
------------------- test 완료 : , 2790 , 256630 , 포인트엔지니어링 ------------------- 
------------------- test 시작_2019-12-5_13:38:33 : , 2791 , 256840 , 한국비엔씨 -------------------  
O_1. ratio_test 통과 2791 , 256840 , 한국비엔씨
O_2. total_volume_test 통과 2791 , 256840 , 한국비엔씨
O_3. average_volume_test 통과 2791 , 256840 , 한국비엔씨
------------------- test 완료 : , 2791 , 256840 , 한국비엔씨 ------------------- 
------------------- test 시작_2019-12-5_13:38:34 : , 2792 , 256940 , 케이피에스 -------------------  
O_1. r

O_1. ratio_test 통과 2823 , 267790 , 배럴
O_2. total_volume_test 통과 2823 , 267790 , 배럴
O_3. average_volume_test 통과 2823 , 267790 , 배럴
------------------- test 완료 : , 2823 , 267790 , 배럴 ------------------- 
------------------- test 시작_2019-12-5_13:38:50 : , 2824 , 267980 , 매일유업 -------------------  
O_1. ratio_test 통과 2824 , 267980 , 매일유업
X_2. total_volume_test 없음 2824 , 267980 , 매일유업
------------------- test 시작_2019-12-5_13:38:51 : , 2825 , 268600 , 셀리버리 -------------------  
O_1. ratio_test 통과 2825 , 268600 , 셀리버리
O_2. total_volume_test 통과 2825 , 268600 , 셀리버리
O_3. average_volume_test 통과 2825 , 268600 , 셀리버리
------------------- test 완료 : , 2825 , 268600 , 셀리버리 ------------------- 
------------------- test 시작_2019-12-5_13:38:51 : , 2826 , 269620 , 시스웍 -------------------  
O_1. ratio_test 통과 2826 , 269620 , 시스웍
O_2. total_volume_test 통과 2826 , 269620 , 시스웍
O_3. average_volume_test 통과 2826 , 269620 , 시스웍
------------------- test 완료 : , 2826 , 269620 , 시스웍 ------------------- 
--------------

O_1. ratio_test 통과 2858 , 290660 , 네오펙트
O_2. total_volume_test 통과 2858 , 290660 , 네오펙트
X_3. average_volume_test 없음 2858 , 290660 , 네오펙트
------------------- test 시작_2019-12-5_13:39:8 : , 2859 , 290670 , 대보마그네틱 -------------------  
O_1. ratio_test 통과 2859 , 290670 , 대보마그네틱
O_2. total_volume_test 통과 2859 , 290670 , 대보마그네틱
X_3. average_volume_test 없음 2859 , 290670 , 대보마그네틱
------------------- test 시작_2019-12-5_13:39:9 : , 2860 , 290720 , 푸드나무 -------------------  
O_1. ratio_test 통과 2860 , 290720 , 푸드나무
X_2. total_volume_test 없음 2860 , 290720 , 푸드나무
------------------- test 시작_2019-12-5_13:39:9 : , 2861 , 290740 , 액트로 -------------------  
O_1. ratio_test 통과 2861 , 290740 , 액트로
O_2. total_volume_test 통과 2861 , 290740 , 액트로
O_3. average_volume_test 통과 2861 , 290740 , 액트로
------------------- test 완료 : , 2861 , 290740 , 액트로 ------------------- 
------------------- test 시작_2019-12-5_13:39:10 : , 2862 , 291210 , 한국제7호스팩 -------------------  
X_1. ratio_test 없음 2862 , 291210 , 한국제7호스팩
---------

X_1. ratio_test 없음 2895 , 313750 , 유안타제4호스팩
------------------- test 시작_2019-12-5_13:39:27 : , 2896 , 313760 , 윌링스 -------------------  
O_1. ratio_test 통과 2896 , 313760 , 윌링스
O_2. total_volume_test 통과 2896 , 313760 , 윌링스
O_3. average_volume_test 통과 2896 , 313760 , 윌링스
------------------- test 완료 : , 2896 , 313760 , 윌링스 ------------------- 
------------------- test 시작_2019-12-5_13:39:28 : , 2897 , 317030 , 케이비17호스팩 -------------------  
X_1. ratio_test 없음 2897 , 317030 , 케이비17호스팩
------------------- test 시작_2019-12-5_13:39:28 : , 2898 , 317120 , 라닉스 -------------------  
O_1. ratio_test 통과 2898 , 317120 , 라닉스
O_2. total_volume_test 통과 2898 , 317120 , 라닉스
X_3. average_volume_test 없음 2898 , 317120 , 라닉스
------------------- test 시작_2019-12-5_13:39:29 : , 2899 , 317240 , 하이제4호스팩 -------------------  
X_1. ratio_test 없음 2899 , 317240 , 하이제4호스팩
------------------- test 시작_2019-12-5_13:39:29 : , 2900 , 317320 , 한화에스비아이스팩 -------------------  
O_1. ratio_test 통과 2900 , 317320 , 한화에스비아이스팩
O_2. 

O_1. ratio_test 통과 2936 , 900110 , 이스트아시아홀딩스
O_2. total_volume_test 통과 2936 , 900110 , 이스트아시아홀딩스
O_3. average_volume_test 통과 2936 , 900110 , 이스트아시아홀딩스
------------------- test 완료 : , 2936 , 900110 , 이스트아시아홀딩스 ------------------- 
------------------- test 시작_2019-12-5_13:39:48 : , 2937 , 900120 , 씨케이에이치 -------------------  
O_1. ratio_test 통과 2937 , 900120 , 씨케이에이치
O_2. total_volume_test 통과 2937 , 900120 , 씨케이에이치
X_3. average_volume_test 없음 2937 , 900120 , 씨케이에이치
------------------- test 시작_2019-12-5_13:39:49 : , 2938 , 900250 , 크리스탈신소재 -------------------  
O_1. ratio_test 통과 2938 , 900250 , 크리스탈신소재
O_2. total_volume_test 통과 2938 , 900250 , 크리스탈신소재
O_3. average_volume_test 통과 2938 , 900250 , 크리스탈신소재
------------------- test 완료 : , 2938 , 900250 , 크리스탈신소재 ------------------- 
------------------- test 시작_2019-12-5_13:39:49 : , 2939 , 900260 , 로스웰 -------------------  
O_1. ratio_test 통과 2939 , 900260 , 로스웰
O_2. total_volume_test 통과 2939 , 900260 , 로스웰
O_3. average_volume_test 통과 2939 , 